In [1]:
import dgl 
from dgl.nn.pytorch.softmax import edge_softmax
from dgl.data.utils import load_graphs

Using backend: pytorch


In [2]:
def ori_softmax(g, nf, ef):
    device = nf.device
    g.ndata['h'] = nf
    g.edata['_'] = ef
    g.edata['w'] = edge_softmax(g, g.edata['_'].to(device))

    wh_func = dgl.function.e_mul_u('w', 'h', 'wh')  # 'Weight' x 'Hidden' -> 'Weighted hidden'
    g.apply_edges(func=wh_func)
    _ = g.ndata.pop('h')
    _ = g.edata.pop('w')
    _ = g.edata.pop('_')

    sum_func = dgl.function.sum('m', 'sm')  # 'Weighted hidden' (multiple tensor) -> 'weighted sum' (one tensor)
    g.update_all(message_func=dgl.function.copy_e('wh', 'm'), reduce_func=sum_func)
    soft_max_nf = g.ndata.pop('sm')
    return soft_max_nf

In [3]:
def my_softmax(g, node_feature, edge_feature):
    g.ndata['h'] = node_feature
    g.edata['w'] = edge_softmax(g, edge_feature)
    g.apply_edges(func=dgl.function.e_mul_u('w', 'h', 'wh'))
    g.update_all(message_func=dgl.function.copy_e('wh', 'm'), reduce_func=dgl.function.sum('m', 'sum_m'))
    softmax_node_feature = g.ndata['sum_m']
    return softmax_node_feature

In [4]:
data = load_graphs('../data/gsi2.bin')
graph = data[0][0]